<span style="font-size:24pt; color:blue; font-family: 'Times New Roman'">P-box Notebook 
    <span style="color:green;"> for creating AR6 style P-box data </span>
</span>
<br>
<span style="font-size:12pt; color:black; font-family:Georgia, serif;font-style:italic">by Praveen Kumar and Robert Kopp.</span>
<br>
<br>
<span style="font-family: 'Times New Roman'; font-size:16pt; color:black"> 
    This is a notebook that can be used to create and vizualize P-box data from raw <a href="https://doi.org/10.5194/egusphere-2023-14" style="color:blue; text-decoration:underline;">FACTS 1.0</a>
    output. Workflows and associated *.nc files are created as part of this notebook. 
<br>
<br>
<span style="font-family:'Courier New', monospace; font-size:12pt; color:black"> <b>Background::</b> <br>
    In order to represent deep uncertainty, Probability boxes (p-boxes) can be constructed. These represent an envelope encompassing alternative probability distributions for describing the same variables. Within this notebook, we construct four p-boxes: <strong>1e, 1f, 2e,</strong> and <strong>2f</strong>.
    <br><br>
    <strong>P-box 1e</strong> and <strong>2e</strong> use ice-sheet projections from an emulator that accurately captures the output of both the GlacierMIP glacier model intercomparison and the ISMIP6 ice-sheet model intercomparison.
    <br><br>
    <strong>P-box 1e</strong> and <strong>1f</strong> both use multi-model intercomparison exercises 
    (<a href="https://www.nature.com/articles/s41586-021-03302-y" style="color:blue; text-decoration:underline;">ISMIP6</a> and <a href="https://doi.org/10.5194/esd-11-35-2020" style="color:blue; text-decoration:underline;">LARMIP2</a>). 
    These p-boxes represent a medium-to-high level of agreement among the participating models. The IPCC AR6 thus assessed medium confidence in their projections of included processes.
</span>
<br>
<br>
<br>

<br>
<span style="font-size:24pt; color:blue; font-family: 'Times New Roman';"> Set <b>expFolder</b> ::
    <span style="font-size:14pt; color:green;"> point to where FACTS raw output is located (e.g. /facts/experiments) </span>

</span>


<span style="background-color: yellow;"> need to modify for NZ and generic facts experiments </span> currently configured to the coupling syntax

In [3]:
import os

In [4]:
expFolder="/projects/kopp/facts-experiments/221217"
#
region = 'global'
#
import fun_pbox_Gen_v3 as fn
#
ssps=["ssp119","ssp126","ssp245","ssp370","ssp585"]

<br>
<span style="font-size:24pt; color:blue; font-family: 'Times New Roman';"> Generate::
    <span style="font-size:14pt; color:green;">Quantile files from FACTS output and arrange in workflow/ssp folders </span>

</span>


In [5]:
def copy_and_convert(source_file, dest_folder, region):
    #shutil.copy2(source_file, dest_folder)
    out_file_path = os.path.join(dest_folder, os.path.basename(source_file).split(".nc")[0] + "_quantiles.nc")
    fn.Samples_to_Quantiles(source_file, out_file_path)

path = fn.create_directory("1_workflow","remove")

skip = {
    'wf3e': ['ssp119', 'ssp370'],
    'wf3f': ['ssp119', 'ssp370'],
    'wf4':  ['ssp119', 'ssp245', 'ssp370']
}

for workflow, file_patterns in fn.WF_file_patterns.items():
    for ssp in ssps:
        if ssp in skip.get(workflow, []):
            continue
        
        ssp_path = os.path.join(path, workflow, ssp)
        os.makedirs(ssp_path, exist_ok=True)
        
        # Copy Component files & convert to Quantiles.
        for file in file_patterns:
            component_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{file}_{region}sl.nc'
            copy_and_convert(component_file_path, ssp_path, region)
        
        # Copy Common files & convert to Quantiles.
        for common_file_item in fn.common_files:
            common_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{common_file_item}_{region}sl.nc'
            copy_and_convert(common_file_path, ssp_path, region)
        
        # Copy Total files & convert to Quantiles.
        total_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.total.workflow.{workflow}.{region}.nc'
        copy_and_convert(total_file_path, ssp_path, region)
        
        # Copy VLM file & convert to Quantiles.  
        if region == 'local':
            vlm_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{fn.local_files}.{region}sl.nc'
            copy_and_convert(vlm_file_path, ssp_path, region)


<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Pbox files and folders</li>
</ol>  
</span>

In [10]:
copy_file_with_pattern = lambda ssp, wf, pattern: fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")
copy_all_wf_files = lambda p, s, d: fn.copy_all_files_from(os.path.join(os.getcwd(), f'1_workflow/wf{p.split("_")[1]}/{s}'), d)
#
def process_pb_1(pbox, ssp_path, ssp):
    patterns = {
        'pb_1e': ("emulandice.AIS", "larmip.AIS", 'e'),
        'pb_1f': ("ipccar5.icesheets_AIS", "larmip.AIS", 'f')
    }
    pattern1, pattern2, wf_suffix = patterns[pbox]
    # AIS Pbox component
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, pattern)}' for pattern in (pattern1, pattern2)]
    outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    
    # Total
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, f"total.workflow.wf{i}{wf_suffix}")}' for i in (1, 2)]
    outfile = f'{ssp_path}/total-workflow.nc'
    print(infiles)
#     fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
#
path = fn.create_directory("2_pbox","remove")
#
for pbox in fn.PB_file_patterns:
    os.makedirs(os.path.join(path, pbox), exist_ok=True)
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = fn.create_directory(f'{os.path.join(path, pbox,ssp)}')
        # 
        if pbox in ['pb_1e']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            copy_file_with_pattern(ssp, 'wf2e', 'larmip.AIS')
            #
            if pbox in ['pb_1e']:
                process_pb_1(pbox, ssp_path, ssp)

['/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp119/coupling.ssp119.total.workflow.wf1e.global_quantiles.nc', '/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp119/None']
['/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp126/coupling.ssp126.total.workflow.wf1e.global_quantiles.nc', '/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp126/None']
['/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp245/coupling.ssp245.total.workflow.wf1e.global_quantiles.nc', '/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp245/None']
['/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/up

In [7]:
copy_file_with_pattern = lambda ssp, wf, pattern: fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")
copy_all_wf_files = lambda p, s, d: fn.copy_all_files_from(os.path.join(os.getcwd(), f'1_workflow/wf{p.split("_")[1]}/{s}'), d)
#
def process_pb_1(pbox, ssp_path, ssp):
    patterns = {
        'pb_1e': ("emulandice.AIS", "larmip.AIS", 'e'),
        'pb_1f': ("ipccar5.icesheets_AIS", "larmip.AIS", 'f')
    }
    pattern1, pattern2, wf_suffix = patterns[pbox]
    # AIS Pbox component
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, pattern)}' for pattern in (pattern1, pattern2)]
    outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    
    # Total
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, f"total.workflow.wf{i}{wf_suffix}")}' for i in (1, 2)]
    outfile = f'{ssp_path}/total-workflow.nc'
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
#
#    
def handle_component(ssp_path, patterns, outfile_prefix, pbox, ssp):
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, pattern)}' for pattern in patterns]
    outfile = f"{ssp_path}/{outfile_prefix}-pb{pbox.split('_')[1]}-{outfile_prefix.split('-')[1]}-{ssp}_globalsl.nc"
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    
configurations = {
            'pb_2e': {
                'copy_patterns': [('wf1e', 'emulandice.AIS'), ('wf3e', 'deconto21.AIS'), 
                                  ('wf4', 'bamber19.icesheets_AIS'), ('wf4', 'bamber19.icesheets_GIS'), 
                                  ('wf1f', 'ipccar5.glaciers')],
                'ais_patterns': ["emulandice.AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"],
                'gis_patterns': ["emulandice.GrIS", "bamber19.icesheets_GIS"],
                'glacier_patterns': ["emulandice.glaciers", "ipccar5.glaciers"],
                'total_patterns': ["total.workflow.wf2e", "total.workflow.wf1e", "total.workflow.wf3e", "total.workflow.wf4"]
            },
            'pb_2f': {
                'copy_patterns': [('wf1f', 'ipccar5.icesheets_AIS*'), ('wf3f', 'deconto21.AIS*'), 
                                  ('wf4', 'bamber19.icesheets_AIS*'), ('wf4', 'bamber19.icesheets_GIS')],
                'ais_patterns': ["ipccar5.icesheets_AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"],
                'gis_patterns': ["GrIS1f.FittedISMIP.GrIS", "bamber19.icesheets_GIS"],
                'total_patterns': ["total.workflow.wf1f", "total.workflow.wf2f", "total.workflow.wf3f", "total.workflow.wf4"]
            }
}    
    
    
#
path = fn.create_directory("2_pbox","remove")
#
for pbox in fn.PB_file_patterns:
    os.makedirs(os.path.join(path, pbox), exist_ok=True)
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = fn.create_directory(f'{os.path.join(path, pbox,ssp)}')
        # 
        if pbox in ['pb_1e','pb_1f']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            copy_file_with_pattern(ssp, 'wf2e', 'larmip.AIS')
            #
            if pbox in ['pb_1e', 'pb_1f']:
                process_pb_1(pbox, ssp_path, ssp)
        #
        #
        #
        # ---> shrink below
        if pbox in configurations:
            config = configurations[pbox]
            copy_all_wf_files(pbox, ssp, ssp_path)
            #
            for suffix, pattern in config['copy_patterns']:
                copy_file_with_pattern(ssp, suffix, pattern)
            #
            handle_component(ssp_path, config['ais_patterns'], "icesheets-AIS", pbox, ssp)
            handle_component(ssp_path, config['gis_patterns'], "icesheets-GIS", pbox, ssp)
            if pbox in ['pb_2e']: handle_component(ssp_path, config['glacier_patterns'], "glaciers", pbox, ssp)
            handle_component(ssp_path, config['total_patterns'], "total-workflow", pbox, ssp)

            # Uncomment below if you want to delete files
            # for pattern1, pattern2 in [("*AIS*", "icesheets-pb*"), ("*GIS*", "icesheets-pb*"), ("*glaciers*", "glaciers-pb*"), ("*total*", "*-workflow*")]:
            #     fn.delete_files_with_pattern(ssp_path, pattern1, pattern2)
            
        

FileNotFoundError: [Errno 2] No such file or directory: b'/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/updated_GMD_nbk/notebooks_OG/P-box/2_pbox/pb_1e/ssp119/None'

### Old version

In [ ]:
copy_file_with_pattern = lambda ssp, wf, pattern: fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")
copy_all_wf_files = lambda p, s, d: fn.copy_all_files_from(os.path.join(os.getcwd(), f'1_workflow/wf{p.split("_")[1]}/{s}'), d)
#
#
path = fn.create_directory("2_pbox_old_version","remove")
#
for pbox in fn.PB_file_patterns:
    os.makedirs(os.path.join(path, pbox), exist_ok=True)
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = fn.create_directory(f'{os.path.join(path, pbox,ssp)}')
        #
        # 
        if pbox in ['pb_1e','pb_1f']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # also copy
            copy_file_with_pattern(ssp, 'wf2e', 'larmip.AIS')
            #
            #
            if pbox in ['pb_1e']:
                #
                # AIS Pbox component
                fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                copy_file_with_pattern(ssp, 'wf2e', 'total.workflow')
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1e");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                #fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                #fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
            #
            #
            if pbox in ['pb_1f']:
                #
                # AIS Pbox component.
                fname1=fn.find_filename_with_pattern(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                #
                outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                # total
                copy_file_with_pattern(ssp, 'wf2f', 'total.workflow')
                #
                fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1f");               
                fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      
                outfile=f'{ssp_path}/total-workflow.nc'
                fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                #fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                #fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
        #
        #
        if pbox in ['pb_2e']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # Also copy
            copy_file_with_pattern(ssp, 'wf1e', 'emulandice.AIS')
            copy_file_with_pattern(ssp, 'wf3e', 'deconto21.AIS')
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_AIS') 
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_GIS')
            copy_file_with_pattern(ssp, 'wf1f', 'ipccar5.glaciers')
            #
            # AIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.AIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
            fname3=fn.find_filename_with_pattern(ssp_path,"deconto21.AIS")
            fname4=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #   
            # GIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.GrIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   
            #
            # Glaciers Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"emulandice.glaciers")
            fname2=fn.find_filename_with_pattern(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/glaciers-pb{pbox.split('_')[1]}-glaciers-{ssp}_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            # total
            copy_file_with_pattern(ssp, 'wf1e', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf3e', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf4' , 'total.workflow') 
            fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2e");               
            fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1e"); 
            fname3=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf3e"); 
            fname4=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}']; 
            outfile=f'{ssp_path}/total-workflow.nc'
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            #fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            #fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            #fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            #fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")
        #
        #        
        if pbox in ['pb_2f']:
            copy_all_wf_files(pbox, ssp, ssp_path) 
            # Also copy
            copy_file_with_pattern(ssp, 'wf1f', 'ipccar5.icesheets_AIS*');
            copy_file_with_pattern(ssp, 'wf3f', 'deconto21.AIS*')
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_AIS*');
            copy_file_with_pattern(ssp, 'wf4' , 'bamber19.icesheets_GIS')
            #
            # AIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"larmip.AIS")
            fname3=fn.find_filename_with_pattern(ssp_path,"deconto21.AIS")
            fname4=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_AIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #   
            # GIS Pbox component
            fname1=fn.find_filename_with_pattern(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.find_filename_with_pattern(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile = f"{ssp_path}/icesheets-pb{pbox.split('_')[1]}-icesheets-{ssp}_GIS_globalsl.nc"
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   
            #
            # total
            copy_file_with_pattern(ssp, 'wf1f', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf3f', 'total.workflow')
            copy_file_with_pattern(ssp, 'wf4' , 'total.workflow') 
            fname1=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf1f");               
            fname2=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf2f"); 
            fname3=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf3f"); 
            fname4=fn.find_filename_with_pattern(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}']; 
            outfile=f'{ssp_path}/total-workflow.nc'
            fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            #fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            #fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            #fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")            

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Confidence level files files and folders</li>
</ol>  
</span>

In [ ]:
# path = fn.create_directory("3_confidence_level_files",,"remove")
# fn.generate_confidence_files(os.getcwd()+'/2_pbox', path)